In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gzip -d cc.en.300.vec.gz

!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
!gzip -d cc.fr.300.vec.gz

--2023-04-18 17:11:47--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.82, 13.35.7.50, 13.35.7.128, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G   153MB/s    in 8.7s    

2023-04-18 17:11:56 (145 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]

--2023-04-18 17:12:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.82, 13.35.7.38, 13.35.7.128, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1287757366 (1.2G) [binary/octet-stream]
Saving to: ‘cc.fr.300.vec.gz’

cc.f

In [6]:
from gensim.models import KeyedVectors
import numpy as np


en_emb = KeyedVectors.load_word2vec_format("/kaggle/working/cc.en.300.vec")
fr_emb = KeyedVectors.load_word2vec_format("/kaggle/working/cc.fr.300.vec")

In [7]:
august_embedding = en_emb["august"]
august_embedding.shape, august_embedding[:5]

((300,), array([-0.0522,  0.0364, -0.1252,  0.0053,  0.0382], dtype=float32))

In [8]:
en_emb.most_similar([august_embedding], topn=3)

[('august', 0.9999999403953552),
 ('september', 0.8252838850021362),
 ('october', 0.8111193180084229)]

In [9]:
en_emb[["august", "september"]].shape

(2, 300)

In [10]:
fr_emb.most_similar([fr_emb["aout"]])

[('aout', 1.0),
 ('Aout', 0.8249964118003845),
 ('juillet', 0.8109882473945618),
 ('fevrier', 0.8072442412376404),
 ('septembre', 0.7838520407676697),
 ('août', 0.779176652431488),
 ('juin', 0.7692081332206726),
 ('octobre', 0.7597455382347107),
 ('decembre', 0.7595790028572083),
 ('avril', 0.7390779256820679)]

### Translation

In [11]:
def load_word_pairs(filename):
    en_fr_pairs = []
    en_vectors = []
    fr_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            en, fr = line.rstrip().split(" ")
            if en not in en_emb or fr not in fr_emb:
                continue
            en_fr_pairs.append((en, fr))
            en_vectors.append(en_emb[en])
            fr_vectors.append(fr_emb[fr])
    return en_fr_pairs, np.array(en_vectors), np.array(fr_vectors)

In [12]:
!wget -O en-fr.train.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.train.txt
!wget -O en-fr.test.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.test.txt

--2023-04-18 17:24:35--  https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 178608 (174K) [text/plain]
Saving to: ‘en-fr.train.txt’

en-fr.train.txt     100%[===================>] 174.42K  --.-KB/s    in 0.03s   

2023-04-18 17:24:36 (5.87 MB/s) - ‘en-fr.train.txt’ saved [178608/178608]

--2023-04-18 17:24:37--  https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response...

In [13]:
en_fr_train, X_train, Y_train = load_word_pairs("/kaggle/working/en-fr.train.txt")
en_fr_test, X_test, Y_test = load_word_pairs("/kaggle/working/en-fr.test.txt")

### Embedding space mapping

In [14]:
from sklearn.linear_model import LinearRegression


# YOUR CODE HERE
mapping = LinearRegression()
mapping.fit(X_train, Y_train)

LinearRegression()

In [15]:
august = mapping.predict(en_emb["august"].reshape(1, -1))
fr_emb.most_similar(august)

[('aout', 0.7535635232925415),
 ('juin', 0.749801516532898),
 ('juillet', 0.7457543611526489),
 ('septembre', 0.7445200085639954),
 ('mars', 0.7354801893234253),
 ('octobre', 0.7344686985015869),
 ('novembre', 0.7261154055595398),
 ('février', 0.7228148579597473),
 ('janvier', 0.721759557723999),
 ('avril', 0.7185364365577698)]

In [16]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    count = 0
    length = len(pairs)
    for i in range(length):
        pair = pairs[i]
        predicted_vector = mapped_vectors[i]
        for sim in fr_emb.most_similar([predicted_vector], topn=topn):
            if sim[0] == pair[1]:
                count += 1
    return count/length
        
    

In [17]:
assert precision([("august", "aout")], august, topn=5) == 1.0
assert precision([("august", "aout")], august, topn=9) == 1.0
assert precision([("august", "aout")], august, topn=10) == 1.0

In [18]:
assert precision(en_fr_test[:100], X_test[:100]) == 0.0
assert precision(en_fr_test[:100], Y_test[:100]) == 1.0

In [19]:
precision_top1 = precision(en_fr_test[:100], mapping.predict(X_test[:100]), 1)
precision_top5 = precision(en_fr_test[:100], mapping.predict(X_test[:100]), 5)

In [20]:
precision_top1, precision_top5

(0.37, 0.67)

### Making it better (orthogonal Procrustean problem)

It can be shown that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$(W^T)^*= \arg\min_{W^T} \|XW^T - Y\|_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$(W^T)^*=UV^T$$

In [36]:
def svd(x, Y):
    XTY = x.T @ Y
    U, _, Vt = np.linalg.svd(XTY)
    return np.matmul(U, Vt)

mapping_svd = svd(X_train, Y_train)

In [37]:
fr_emb.most_similar([np.matmul(en_emb['.'], mapping_svd)], topn=5)#[0][0]

[('.', 0.5472625494003296),
 ('..', 0.4910860061645508),
 ('!', 0.4725416302680969),
 ('Et', 0.46604833006858826),
 ('Il', 0.4368574023246765)]

### Unsupervised embedding-based MT

In [51]:
import math
import nltk
from nltk.tokenize import WordPunctTokenizer

def find_nearest_word(embedding):
    return fr_emb.most_similar([np.matmul(embedding, mapping_svd)], topn=1)[0][0]

embeddings_dict_en = {word: embedding for word,embedding  in zip(en_emb.key_to_index, en_emb.vectors)}
embeddings_dict_fr = {word: embedding for word,embedding  in zip(fr_emb.key_to_index, fr_emb.vectors)}


def translate(sentence):
    """
    :args:
        sentence - sentence in English (str)
    :returns:
        translation - sentence in French (str)

    * find english embedding for each word in sentence
    * transform english embedding vector
    * find nearest french word and replace
    """
    tokenizer = WordPunctTokenizer()
    translated = []
    tokens = tokenizer.tokenize(sentence)
    # for each word in the sentence, find its embedding in English and transform the vector
    # then find the nearest French word and replace the English word with the French word
    for token in tokens:
        english_word_embedding = embeddings_dict_en.get(token, None)
#         transformed_embedding = transform(english_word_embedding)
        if english_word_embedding is not None:
            french_word = find_nearest_word(english_word_embedding)
            translated.append(french_word)
    # YOUR CODE HERE
    return " ".join(translated)

In [53]:
translate('London is the capital of Greate Britain')

'Londres est dans capital de Morski Grande-Bretagne'

In [54]:
assert translate(".") == "."
assert translate("I walk around Paris") == "je marcher autour Paris"

In [55]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples
import re
import string

nltk.download('twitter_samples')
nltk.download('stopwords')

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        # if (word not in stopwords_english and  # remove stopwords
        #     word not in string.punctuation):  # remove punctuation
        if word not in string.punctuation:
            tweets_clean.append(word)
            # stem_word = stemmer.stem(word)  # stemming word
            # tweets_clean.append(stem_word)

    return " ".join(tweets_clean)

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
twitter_samples.strings('positive_tweets.json')[10:15]

['#FollowFriday @wncer1 @Defense_gouv for being top influencers in my community this week :)',
 "Who Wouldn't Love These Big....Juicy....Selfies :) - http://t.co/QVzjgd1uFo http://t.co/oWBL11eQRY",
 '@Mish23615351  follow @jnlazts &amp; http://t.co/RCvcYYO0Iq follow u back :)',
 "@jjulieredburn Perfect, so you already know what's waiting for you :)",
 'Great new opportunity for junior triathletes aged 12 and 13 at the Gatorade series! Get your entries in :) http://t.co/of3DyOzML0']

In [57]:
for i in twitter_samples.strings('positive_tweets.json')[10:15]:
    print(i, process_tweet(i), sep='\n\n', end='\n-----------------\n')

#FollowFriday @wncer1 @Defense_gouv for being top influencers in my community this week :)

followfriday for being top influencers in my community this week :)
-----------------
Who Wouldn't Love These Big....Juicy....Selfies :) - http://t.co/QVzjgd1uFo http://t.co/oWBL11eQRY

who wouldn't love these big ... juicy ... selfies :)
-----------------
@Mish23615351  follow @jnlazts &amp; http://t.co/RCvcYYO0Iq follow u back :)

follow
-----------------
@jjulieredburn Perfect, so you already know what's waiting for you :)

perfect so you already know what's waiting for you :)
-----------------
Great new opportunity for junior triathletes aged 12 and 13 at the Gatorade series! Get your entries in :) http://t.co/of3DyOzML0

great new opportunity for junior triathletes aged 12 and 13 at the gatorade series get your entries in :)
-----------------


In [58]:
for i in twitter_samples.strings('positive_tweets.json')[:10]:
    print(i, process_tweet(i), translate(process_tweet(i)), sep='\n\n', end='\n-----------------\n')

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

followfriday for being top engaged members in my community this week :)

twitt pour être top engagé membres dans mon communauté cette semaine okk
-----------------
@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!

hey james how odd :/ please call our contact centre on 02392441234 and we will be able to assist you :) many thanks

hey christopher comment bizarre veuillez appeler notre contacter centre sur et nous pourra être puisse amener aider vous okk nombreux merci
-----------------
@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!

we had a listen last night :) as you bleed is an amazing track when are you in scotland

nous avait un écouter dernière nuit okk comme vous saigner est un incroyable track quand sont vous dans ecosse
-----------